In [ ]:
#source.
# https://docs.ultralytics.com/tasks/classify/

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d pratik2901/multiclass-weather-dataset

100% 91.4M/91.4M [00:00<00:00, 152MB/s]
100% 91.4M/91.4M [00:00<00:00, 126MB/s]


In [ ]:
import zipfile
zip_ref =zipfile.ZipFile('/content/multiclass-weather-dataset.zip','r');
zip_ref.extractall('/content');
zip_ref.close();

In [ ]:
#strucking the dataset in yolov8 format
import os
import random
import shutil

# Path to your dataset folder
dataset_path = "/content/Multi-class Weather Dataset"

# Paths for train and test folders
train_path = os.path.join(dataset_path, "train")
test_path = os.path.join(dataset_path, "test")

# Create train and test folders if they don't exist
os.makedirs(train_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)

# List of class folders
class_folders = ["Cloudy", "Rain", "Shine", "Sunrise"]

# Percentage split between train and test
train_split = 0.6
test_split = 0.4

# Create class folders within train and test folders
for folder in class_folders:
    train_class_folder = os.path.join(train_path, folder)
    test_class_folder = os.path.join(test_path, folder)
    os.makedirs(train_class_folder, exist_ok=True)
    os.makedirs(test_class_folder, exist_ok=True)

# Iterate through each class folder
for class_folder in class_folders:
    class_folder_path = os.path.join(dataset_path, class_folder)
    class_images = os.listdir(class_folder_path)

    # Shuffle the image list
    random.shuffle(class_images)

    # Calculate split indices
    split_index = int(len(class_images) * train_split)

    # Split images into train and test sets
    train_images = class_images[:split_index]
    test_images = class_images[split_index:]

    # Copy images to class folders within train and test
    for image in train_images:
        src_path = os.path.join(class_folder_path, image)
        dst_path = os.path.join(train_path, class_folder, image)
        shutil.copy(src_path, dst_path)

    for image in test_images:
        src_path = os.path.join(class_folder_path, image)
        dst_path = os.path.join(test_path, class_folder, image)
        shutil.copy(src_path, dst_path)


In [ ]:
!pip install ultralytics
import numpy as np;
from ultralytics import YOLO

In [ ]:
model = YOLO('yolov8n-cls.pt')  # load a pretrained model (recommended for training)

In [ ]:
import shutil

# Path to the folder you want to delete
folder_path = "/content/Multi-class Weather Dataset/Shine"

# Use shutil.rmtree to remove the folder and its contents
shutil.rmtree(folder_path)


In [ ]:
#training model using training and validation dataset.
results = model.train(data='/content/Multi-class Weather Dataset', epochs=15, imgsz=64)

Ultralytics YOLOv8.0.165 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=/content/Multi-class Weather Dataset, epochs=15, patience=50, batch=16, imgsz=64, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, ker

In [ ]:
#use last.pt pretrained model to predict the unseen data and check model performance.
inference=YOLO('/content/runs/classify/train4/weights/last.pt');

In [ ]:
results=inference.predict('/content/Multi-class Weather Dataset/val/Cloudy/cloudy100.jpg',save=True);


image 1/1 /content/Multi-class Weather Dataset/val/Cloudy/cloudy100.jpg: 64x64 Cloudy 1.00, Shine 0.00, Sunrise 0.00, Rain 0.00, 3.4ms
Speed: 0.5ms preprocess, 3.4ms inference, 0.1ms postprocess per image at shape (1, 3, 64, 64)
Results saved to runs/classify/predict2


In [ ]:
print(results);

[ultralytics.engine.results.Results object with attributes:

boxes: None
keypoints: None
masks: None
names: {0: 'Cloudy', 1: 'Rain', 2: 'Shine', 3: 'Sunrise'}
orig_img: array([[[145, 135, 135],
        [148, 138, 138],
        [155, 143, 141],
        ...,
        [172, 146, 130],
        [167, 148, 135],
        [171, 152, 139]],

       [[150, 136, 137],
        [153, 140, 138],
        [159, 144, 142],
        ...,
        [172, 143, 128],
        [169, 146, 131],
        [172, 148, 136]],

       [[163, 139, 139],
        [164, 141, 139],
        [166, 143, 141],
        ...,
        [172, 138, 122],
        [169, 140, 125],
        [172, 143, 129]],

       ...,

       [[ 50,  98, 104],
        [ 21,  69,  73],
        [ 44,  90,  97],
        ...,
        [ 47,  81,  94],
        [ 70, 104, 117],
        [ 50,  84,  97]],

       [[ 63, 104, 113],
        [ 13,  55,  62],
        [ 38,  77,  86],
        ...,
        [ 30,  65,  79],
        [ 38,  73,  87],
        [ 39,  74,  

In [ ]:
#now check model performace

results=inference.predict('/content/download.jpg',save=True);


image 1/1 /content/download.jpg: 64x64 Rain 0.75, Shine 0.21, Cloudy 0.04, Sunrise 0.00, 4.1ms
Speed: 0.7ms preprocess, 4.1ms inference, 0.1ms postprocess per image at shape (1, 3, 64, 64)
Results saved to runs/classify/predict2


In [ ]:
names_dict=results[0].names

<class 'numpy.ndarray'>


IndexError: ignored

In [ ]:
print(probs)

ultralytics.engine.results.Probs object with attributes:

data: tensor([3.9469e-02, 7.4743e-01, 2.1293e-01, 1.6741e-04], device='cuda:0')
orig_shape: None
shape: torch.Size([4])
top1: 1
top1conf: tensor(0.7474, device='cuda:0')
top5: [1, 2, 0, 3]
top5conf: tensor([7.4743e-01, 2.1293e-01, 3.9469e-02, 1.6741e-04], device='cuda:0')


In [ ]:
#build-in and pretrained on large imagenet dataset.
models = YOLO('yolov8n-cls.pt')  # load a pretrained model (recommended for training)

In [ ]:
results=model.predict('/content/download.jpg',save=True);


image 1/1 /content/download.jpg: 64x64 Rain 0.75, Shine 0.21, Cloudy 0.04, Sunrise 0.00, 23.9ms
Speed: 3.9ms preprocess, 23.9ms inference, 0.1ms postprocess per image at shape (1, 3, 64, 64)
Results saved to runs/classify/predict3
